In [1]:
%matplotlib inline

import numpy as np
import math
import os
import matplotlib
import matplotlib.pyplot as plt
import librosa as lb
import librosa.display as lbd
import IPython.display as ipd
import scipy.signal as ss
import scipy.spatial as sspat
import scipy.fft as sfft
import warnings
import pickle as pkl
import time

In [2]:
%cd ../
import CythonDTW
import TSM_Import

/home/krong/Documents/classes/SP/final/AdaptiveAccompaniment


In [3]:
hop_length = 512

In [4]:
fileId_path = "experiments/fileIds.pkl"
queryInfo_path = "experiments/queryInfo.pkl"
timeData_path = "experiments/timeData.pkl"
audioFiles_path = "experiments/audioFiles.pkl"
with open(fileId_path, 'rb') as f:
    fileIds=pkl.load(f)
with open(queryInfo_path, 'rb') as f:
    queryInfo=pkl.load(f)
with open(timeData_path, 'rb') as f:
    timeData=pkl.load(f)
with open(audioFiles_path, 'rb') as f:
    audioFiles=pkl.load(f)

In [5]:
sr = 22050

In [6]:
def getChromaFeatures(audio):
    chroma = lb.feature.chroma_stft(audio, norm=2)
    return chroma

In [7]:
def subsequencialDTW(cost_matrix):
    steps = np.array([2, 1, 1, 2, 1, 1]).reshape((3,2))
    weights = [2,1,1]
    D,wp = lb.sequence.dtw(C=cost_matrix, step_sizes_sigma=steps, weights_mul=weights, subseq=True, backtrack=True)
    return D,wp

In [8]:
def getCostMatrix(query, ref):
    return sspat.distance.cdist(query.T, ref.T, metric='cosine')

In [9]:
def inverse_time_function(wp):
    wp = wp*512
    output = np.zeros((wp[-1,0])+1)
    for pair_index in range(len(wp)-1):
        x_0 = wp[pair_index,0]
        x_1 = wp[pair_index+1,0]
        y_0 = wp[pair_index,1]
        y_1 = wp[pair_index+1,1]
        slope = (y_1-y_0)/(x_1-x_0)
        array = y_0+np.arange(x_1-x_0)*slope
        output[x_0:x_1] =array.astype(int)
    output[-1] = wp[-1,1]
    return output

In [10]:
def tf_preprocessing(local_paths):
    ls = []
    for seg in local_paths:
        new_seg = []
        for pair in seg:
            new_seg.append(np.array([pair[0],pair[1]]))
        ls.append(np.array(new_seg))
    return ls
def seg_time_function(wp):
    start_time  = time.time()
    wp = wp*hop_length
    output = np.zeros((wp[-1,1]-wp[0,1])+1)
    x_b = wp[0,1]
    for pair_index in range(len(wp)-1):
        y_0 = wp[pair_index,0]
        y_1 = wp[pair_index+1,0]
        x_0 = wp[pair_index,1]
        x_1 = wp[pair_index+1,1]
        slope = (y_1-y_0)/(x_1-x_0)
        array = y_0+np.arange(x_1-x_0)*slope
        output[x_0-x_b:x_1-x_b] = array
    output[-1] = wp[-1,0]

    return output
def group_time_function(wp,ref_audio_len):
    wp = tf_preprocessing(wp)
    start_time = time.time()

    output = np.zeros(ref_audio_len)
    

    current_start = 0
    for index, seg_path in enumerate(wp):

        pre_padd_end = seg_path[0][1] * hop_length

        output[current_start:pre_padd_end]=(np.arange(pre_padd_end-current_start)+1)+output[current_start-1]

        current_start = pre_padd_end
        tf = seg_time_function(seg_path)
        if index > 0:
            start_point = max(0,wp[index-1][-1][1]-wp[index][0][1])*hop_length
            end_point = min(len(output)-current_start,len(tf))

            tf = tf[start_point:end_point]

        if len(tf)>=2:
            output[current_start:current_start+len(tf)] = tf+tf[1]+output[current_start-1]
        elif len(tf)==1:
            output[current_start:current_start+len(tf)] = tf+output[current_start-1]
        current_start = current_start+len(tf)

    output[current_start:]=np.arange(ref_audio_len-current_start)+output[current_start-1]
    return output

In [11]:
def realTimeAccompaniment(new_input,stored_info,evaluate, sim):
    #prior = stored_info['prior']
    orchestra_chroma = stored_info['orchestra_chroma']
    '''
    audio_input = np.zeros((len(new_input)+len(prior)))
    if len(prior)>0:
        audio_input[:len(prior)]=prior
        audio_input[len(prior):]=new_input
    else:
        audio_input=new_input
    '''

    orchestra_audio = stored_info['orchestra_audio']
    input_chroma = getChromaFeatures(new_input)
    cost_matrix = getCostMatrix(input_chroma, orchestra_chroma)
    D,wp = subsequencialDTW(cost_matrix)
    wp = np.sort(wp,axis=0)
    tf, itf, audio_output = [], [], []
    if evaluate:
        tf = group_time_function([wp],len(orchestra_audio))
    if sim:
        itf = inverse_time_function(wp)
        audio_output = TSM_Import.phase_vocoder(orchestra_audio, (itf).astype(int))
    
    #stored_info['prior'].append(new_input)
    return audio_output, tf, stored_info

In [12]:
def realTimeSim(soloist, orchestra, sr = 22050):
    stored_info = dict()
    accompanyment = []
    orchestra_chroma = getChromaFeatures(orchestra)
    stored_info["orchestra_chroma"]=orchestra_chroma 
    stored_info['orchestra_audio'] = orchestra
    for solo_seg in soloist:
        accompanyment_seg = np.zeros(len(solo_seg))
        #stored_info['prior'] = []
        current_seconds = 0
        input_length_sec = .023*100
        input_length_frame = input_length_sec*sr
        num_input = int(np.ceil(len(solo_seg)/input_length_frame))
        for input_index in range(num_input):
            audio_input = solo_seg[int(input_index*input_length_frame):int((input_index+1)*input_length_frame)]
            start_time = time.time()
            audio_output, time_function_output, stored_info = realTimeAccompaniment(audio_input,stored_info,False, True)
            
            time_offset = time.time()-start_time
            print(time_offset)
            start_point = int(input_index*input_length_frame+time_offset)
            if start_point+len(audio_output)<len(accompanyment_seg):
                accompanyment_seg[start_point:start_point+len(audio_output)]=audio_output
            else:
                accompanyment_seg[start_point:]=audio_output[:len(accompanyment_seg)-start_point]
        accompanyment.append(accompanyment_seg)
    return accompanyment

In [32]:
soloist = audioFiles["p1s_perf12"]
orchestra = audioFiles["p1f_perf16"]
accompanyment = realTimeSim(soloist,orchestra)

/home/krong/Documents/classes/SP/final/AdaptiveAccompaniment/src/TSM_Import.py:128: ComplexWarning: Casting complex values to real discards the imaginary part
  spectrogram[:,index]= spec


w nom (1025, 1) hop (98,) S (1025, 98) analysis_instances (99,) S[0] (98,)
0.2548060417175293
w nom (1025, 1) hop (98,) S (1025, 98) analysis_instances (99,) S[0] (98,)
0.10673379898071289
w nom (1025, 1) hop (98,) S (1025, 98) analysis_instances (99,) S[0] (98,)
0.10602641105651855
w nom (1025, 1) hop (98,) S (1025, 98) analysis_instances (99,) S[0] (98,)
0.10366153717041016
w nom (1025, 1) hop (98,) S (1025, 98) analysis_instances (99,) S[0] (98,)
0.1100931167602539
w nom (1025, 1) hop (98,) S (1025, 98) analysis_instances (99,) S[0] (98,)
0.1100153923034668
w nom (1025, 1) hop (98,) S (1025, 98) analysis_instances (99,) S[0] (98,)
0.10537958145141602
w nom (1025, 1) hop (98,) S (1025, 98) analysis_instances (99,) S[0] (98,)
0.10469865798950195
w nom (1025, 1) hop (98,) S (1025, 98) analysis_instances (99,) S[0] (98,)
0.10859537124633789
w nom (1025, 1) hop (98,) S (1025, 98) analysis_instances (99,) S[0] (98,)
0.10270309448242188
w nom (1025, 1) hop (98,) S (1025, 98) analysis_insta

In [13]:
def get_piece():
    solo_piece = [[] for i in range(4)]
    orch_piece = [[] for i in range(4)]
    for j in timeData:
        if j[2]=="s":
            solo_piece[int(j[1])-1].append(j)
        else:
            orch_piece[int(j[1])-1].append(j)
    return orch_piece, solo_piece

In [14]:
def all_annotation_data(orch_piece, solo_piece):
    solo_seg_player_time = []
    orch_seg_player_time = []
    seg_time_key = [2,4,4,4]
    for piece_id in range(len(solo_piece)):
        piece_i = []
        for i in solo_piece[piece_id]:
            piece_i.append(timeData[i])
        segment = []
        for seg_str in queryInfo["p"+str(piece_id+1)+"s"]:

            comma = seg_str.index(",")
            start, end = seg_str[1:comma],seg_str[comma+1:-1]
            player = []
            #print(start,end)
            for pTimeData in piece_i:
                time = []
                current = start
                while current != end:
                    time.append(pTimeData[current])
                    curr_key = int(current[-1])
                    if curr_key<seg_time_key[piece_id]:
                        current=current[:-1]+str(curr_key+1)
                    else:
                        current=str(int(current[:-2])+1)+'.1'
                    #print(current)
                player.append(time)
            segment.append(player)
        solo_seg_player_time.append(segment)


    for piece_id in range(len(orch_piece)):
        piece_i = []
        for i in orch_piece[piece_id]:
            piece_i.append(timeData[i])
        segment = []
        for seg_str in queryInfo["p"+str(piece_id+1)+"s"]:

            comma = seg_str.index(",")
            start, end = seg_str[1:comma],seg_str[comma+1:-1]
            player = []
            for pTimeData in piece_i:
                time = []
                current = start
                while current != end:
                    time.append(pTimeData[current])
                    curr_key = int(current[-1])
                    if curr_key<seg_time_key[piece_id]:
                        current=current[:-1]+str(curr_key+1)
                    else:
                        current=str(int(current[:-2])+1)+'.1'
                    #print(current)
                player.append(time)
            segment.append(player)
        orch_seg_player_time.append(segment)
    return orch_seg_player_time, solo_seg_player_time

In [132]:
soloist = audioFiles["p1s_perf12"][0][:10*sr]
soloist_long = [soloist[int(i//2)] for i in range(int(len(soloist[:5*sr])*2))]
print(len(soloist_long))
extension = [soloist[int(i)+5*sr] for i in range(len(soloist[5*sr:]))]
print(len(extension))
soloist_long.extend(extension)
print(len(soloist_long))
soloist_long = np.array(soloist_long)
soloist_long

220500
110250
330750


array([ 0.00320435,  0.00320435,  0.00234985, ..., -0.06347656,
       -0.03170776, -0.00238037], dtype=float32)

In [133]:
new_soloist = [.001 for i in range(3*sr)]
new_soloist.extend(soloist)
soloist = new_soloist
ipd.Audio(soloist,rate=sr)

In [134]:
ipd.Audio(soloist_long,rate=sr)

In [131]:
len(soloist_long),len(soloist)

(330750, 286650)

In [135]:
def extrapolateTimeStretchFunction(local_paths, N):
  new_stretch = []
  k = 0
  slope = 1
  new_time = 0
  counter = 0
  for i in range(0,N):
    if k<len(local_paths) and i >= local_paths[k][0][1]*hop_length and i <= local_paths[k][-1][1]*hop_length:
      path = local_paths[k]
      #if i == path[0][1]*hop_length:
        #print(i,new_time)
      if i == path[-1][1]*hop_length:
        #print(i, new_time)    
        k+=1
        slope=1
        counter = 0
      elif i == path[counter][1]*hop_length:
        slope = (path[counter+1][0]-path[counter][0])/(path[counter+1][1]-path[counter][1])
        counter+=1
    else:
      slope=1
    new_time+=slope
    new_stretch.append(int(new_time))
  return new_stretch
def getInverseTimeFunction(time_function):
  inverse_function = []
  counter = 0
  for i in range(time_function[-1]+1):
    while time_function[counter]<i:
      counter+=1
    inverse_function.append(counter)
  return inverse_function

In [136]:
    input_audio = np.array(soloist)
    orchestra_audio = soloist_long
    input_chroma = getChromaFeatures(input_audio)
    orchestra_chroma = getChromaFeatures(orchestra_audio)
    cost_matrix = getCostMatrix(input_chroma, orchestra_chroma)
    D,wp = subsequencialDTW(cost_matrix)
    wp = np.sort(wp,axis=0)
    tf, itf, audio_output = [], [], []
    tf = group_time_function([wp],len(orchestra_audio))
    itf = inverse_time_function(wp)
    audio_output = TSM_Import.phase_vocoder(orchestra_audio, getInverseTimeFunction(tf.astype(int)))

/home/krong/Documents/classes/SP/final/AdaptiveAccompaniment/src/TSM_Import.py:128: ComplexWarning: Casting complex values to real discards the imaginary part
  spectrogram[:,index]= spec


w nom (1025, 1) hop (601,) S (1025, 601) analysis_instances (606,) S[0] (601,)


In [15]:
def evalBeatRealTime(sr=22050):
    error_ls = []
    
    print("start preparation")
    orch_piece, solo_piece  = get_piece()
    orch_seg_player_time, solo_seg_player_time= all_annotation_data(orch_piece, solo_piece)
    diff_seg_player = []
    
    input_num_beat = 4
    
    piece_id = 0
    count=0
    correct = 0
    tolerance = .5
    print("complete preparation")
    
    for piece_id in range(len(orch_seg_player_time)):
        print("new piece",piece_id)
        for seg_id in range(len(orch_seg_player_time[piece_id])):
            print("new seg",seg_id)
            diff_o_s_player = []
            for o_player_id in range(len(orch_seg_player_time[piece_id][seg_id])):
                
                
                orch_piece_id = orch_piece[piece_id][o_player_id]
                orch_audio = audioFiles[orch_piece_id]
                orch_time = solo_seg_player_time[piece_id][seg_id][o_player_id]
                
                
                for s_player_id in range(len(solo_seg_player_time[piece_id][seg_id])):
                    
                    
                    solo_piece_id = solo_piece[piece_id][o_player_id]
                    
                    
                    solo_audio  = audioFiles[solo_piece_id][seg_id]
                    
                    stored_info = dict()
                    orchestra_chroma = getChromaFeatures(orch_audio)
                    stored_info["orchestra_chroma"]=orchestra_chroma 
                    stored_info['orchestra_audio'] = orch_audio
                    stored_info['prior'] = []
                    stored_info["last end"] = 0
                    solo_time = solo_seg_player_time[piece_id][seg_id][s_player_id]
                    ground_truth_baseline_sec = solo_time[0]
                    
                    print("seg",seg_id,"player combo",o_player_id,s_player_id,len(solo_time)-input_num_beat-1)
                    
                    for i in range(len(solo_time)-input_num_beat-1):
                        
                        
                        check_nan_ls = [solo_time[i+j] for j in range(input_num_beat+1)]
                        check_nan_ls.extend([orch_time[i+j] for j in range(input_num_beat+1)])
                        check_nan = [math.isnan(num) for num in check_nan_ls]
                        if True in check_nan:
                            #print("skip")
                            continue
                        
                        audio_input = solo_audio[int(solo_time[i]*sr):int(solo_time[i+input_num_beat]*sr)+1]
                        if len(audio_input)<1:
                            #print("skip")
                            continue
                        start_time = time.time()
                        audio_output, time_function_output, stored_info = realTimeAccompaniment(audio_input,stored_info,True, False)
                        
                        time_offset = time.time()-start_time
                        
                        input_sec = [orch_time[i+j] for j in range(input_num_beat)]
                        
                        input_index = [int(j*sr) for j in input_sec]
                        
                        output_index = np.array([time_function_output[j] for j in input_index])
                        
                        output_sec = output_index/sr#+time_offset
                        
                        if i == 0:
                            output_baseline_sec = output_sec[0]
                        
                        
                        output_sec = output_sec - output_baseline_sec
                        
                        ground_truth_sec = np.array([solo_time[i+j] for j in range(input_num_beat)])
                        
                        ground_truth_sec = ground_truth_sec - ground_truth_baseline_sec
                        
                        error = ground_truth_sec-output_sec
                        
                        error_ls.append(error)
                        
                        count+=4
                        correct += np.sum(np.where(np.abs(error)<tolerance,1,0))
                        if i%20 == 0:
                            print("seg",seg_id,"player combo",o_player_id,s_player_id,i)
                            print(error)
                            print("     ",input_sec,output_sec,ground_truth_sec, time_offset)
                            print()
                            

    clean_error_ls =np.zeros(len(error_ls)*input_num_beat)
    for index, i in enumerate(error_ls):
        clean_error_ls[index*input_num_beat:(index+1)*input_num_beat] = i
    return clean_error_ls

In [16]:
evalBeatRealTime()

start preparation
complete preparation
new piece 0
new seg 0
seg 0 player combo 0 0 25
seg 0 player combo 0 0 0
[ 0.00000000e+00  1.08842713e-10 -1.51170726e-05  1.51171769e-05]
      [5.344157218333334, 6.3434769463333325, 7.344595616, 8.210566893333334] [0.         0.99931973 2.00045351 2.86639456] [0.         0.99931973 2.0004384  2.86640968] 0.22336220741271973

seg 0 player combo 0 0 20
[ 1.17918120e-10  1.51171859e-05 -1.51169615e-05 -3.02341995e-05]
      [23.348420257, 24.215510204, 25.08681783833333, 26.015374149666666] [18.00426304 18.87133787 19.74267574 20.67124717] [18.00426304 18.87135299 19.74266062 20.67121693] 0.10142207145690918

seg 0 player combo 0 1 25
seg 0 player combo 0 1 0
[ 0.         -0.08400605 -0.19960695 -0.30303855]
      [5.344157218333334, 6.3434769463333325, 7.344595616, 8.210566893333334] [0.         0.99931973 2.00045351 2.86639456] [0.         0.91531368 1.80084656 2.56335601] 0.13707232475280762

seg 0 player combo 0 2 25
seg 0 player combo 0 2 0
[

seg 0 player combo 3 3 20
[-3.02344059e-05  9.29674115e-11 -3.02343492e-05 -3.02342676e-05]
      [28.71026455, 29.755510204, 30.816160241666665, 31.793983371] [21.80045351 22.84566893 23.90634921 24.88417234] [21.80042328 22.84566893 23.90631897 24.8841421 ] 0.08408403396606445

seg 0 player combo 3 4 25
seg 0 player combo 3 4 0
[ 0.         -0.26323507 -0.54099773 -0.78689342]
      [6.909841269666667, 8.047936508, 9.227377173, 10.36585034] [0.         1.13814059 2.31755102 3.45600907] [0.         0.87490552 1.77655329 2.66911565] 0.15033721923828125

seg 0 player combo 4 0 25
seg 0 player combo 4 0 0
[0.         0.12439909 0.22388511 0.06668178]
      [14.946137565999999, 15.821043084000001, 16.722690854333333, 17.61525321266667] [0.         0.87492063 1.77655329 2.79972789] [0.         0.99931973 2.0004384  2.86640968] 0.14615631103515625

seg 0 player combo 4 0 20
[ 0.5070068   0.50681784  0.25449735 -0.03962207]
      [32.40196523033333, 33.26420257, 34.13242630366667, 35.1693726

seg 0 player combo 0 3 61
seg 0 player combo 0 3 0
[ 0.         -0.07907785 -0.19251701 -0.33687075]
      [15.207407407666665, 15.975449735666666, 16.766651549333332, 17.608420256999995] [0.         0.76802721 1.55922902 2.40099773] [0.         0.68894936 1.36671202 2.06412698] 0.16524696350097656

seg 0 player combo 0 3 20
[-2.90560847 -3.01981859 -3.1678458  -3.26424792]
      [31.403431595, 32.195389266999996, 32.99841269866667, 33.78952380966666] [16.70680272 17.49877551 18.30181406 19.09292517] [13.80119426 14.47895692 15.13396825 15.82867725] 0.11761736869812012

seg 0 player combo 0 3 40
[-5.06261527 -5.23414966 -5.38134543 -5.49873016]
      [47.034074073999996, 47.83185185166667, 48.619168556333335, 49.419879062666666] [32.29106576 33.08884354 33.87614512 34.67687075] [27.22845049 27.85469388 28.4947997  29.17814059] 0.11104893684387207

seg 0 player combo 0 4 61
seg 0 player combo 0 4 0
[0.         0.06866213 0.11111111 0.10956916]
      [15.207407407666665, 15.9754497356666

seg 0 player combo 3 1 40
[-6.30748299 -6.44783069 -6.56380952 -6.69386243]
      [47.91132275166666, 48.537566137333336, 49.17767195766667, 49.86101284966667] [27.22843537 27.85469388 28.49478458 29.17814059] [20.92095238 21.40686319 21.93097506 22.48427816] 0.08359646797180176

seg 0 player combo 3 1 60
[ -9.50559335  -9.67892668  -9.8526833  -10.0101285 ]
      [61.59508692366666, 62.26492819366667, 62.93646258500001, 63.602509448333336] [40.91219955 41.58204082 42.25356009 42.91963719] [31.4066062  31.90311413 32.40087679 32.90950869] 0.09081625938415527

seg 0 player combo 3 2 61
seg 0 player combo 3 2 0
[ 0.         -0.11369615 -0.22761905 -0.39411943]
      [20.68287226, 21.37182161766667, 22.049584278333338, 22.746999243999998] [0.         0.68893424 1.36671202 2.06412698] [0.         0.5752381  1.13909297 1.67000756] 0.1495211124420166

seg 0 player combo 3 2 20
[-2.86297808 -3.00353742 -3.12435374 -3.24789116]
      [34.48406651566666, 35.161829176, 35.816840514, 36.511549508

seg 1 player combo 1 4 60
seg 1 player combo 2 0 60
seg 1 player combo 2 1 60
seg 1 player combo 2 1 0
[ 0.         -0.13271353 -0.1585034  -0.19758125]
      [47.543537415, 48.171761149, 48.680015117, 49.23088435366666] [0.         0.62820862 1.13646259 1.68730159] [0.         0.49549509 0.97795918 1.48972033] 0.12610888481140137

seg 1 player combo 2 2 60
seg 1 player combo 2 3 60
seg 1 player combo 2 4 60
seg 1 player combo 3 0 60
seg 1 player combo 3 1 60
seg 1 player combo 3 1 0
[ 0.         -0.19520786 -0.34290249 -0.5250189 ]
      [65.69690098266666, 66.387588813, 67.01773242633334, 67.71162509466667] [0.         0.69070295 1.32086168 2.01473923] [0.         0.49549509 0.97795918 1.48972033] 0.14582467079162598

seg 1 player combo 3 2 60
seg 1 player combo 3 3 60
seg 1 player combo 3 4 60
seg 1 player combo 4 0 60
seg 1 player combo 4 1 60
seg 1 player combo 4 1 0
[ 0.         -0.31520786 -0.63773243 -0.96039305]
      [98.15820105833335, 98.96888888900001, 99.77390778533334, 1

...

seg 0 player combo 3 3 40
[ 6.80202561e-11  1.51167188e-05 -1.51173129e-05  2.72102341e-11]
      [27.641602418999998, 28.341300075333333, 29.06063492066667, 29.782690854333335] [27.20707483 27.90675737 28.62612245 29.34816327] [27.20707483 27.90677249 28.62610733 29.34816327] 0.11551976203918457

seg 0 player combo 3 4 41
seg 0 player combo 3 4 0
[0.         0.1928647  0.27773243 0.33177627]
      [0.434527589, 1.0632199543333334, 1.7688737716666667, 2.5031141343333334] [0.         0.62866213 1.33433107 2.06857143] [0.         0.82152683 1.61206349 2.40034769] 0.17549538612365723

seg 0 player combo 3 4 20
[1.76179894 1.77513228 1.87727891 1.95925926]
      [14.289191232, 15.009145880666665, 15.673393802, 16.354391534333335] [13.85464853 14.57460317 15.23886621 15.91986395] [15.61644747 16.34973545 17.11614512 17.8791232 ] 0.09932613372802734

seg 0 player combo 4 0 41
seg 0 player combo 4 0 0
[ 0.         -0.08594104 -0.16385488 -0.17989418]
      [13.858427815666667, 14.679954648333

seg 1 player combo 1 0 20
[1.04663643 1.07070295 1.08920635 1.20196523]
      [58.685729403, 59.447135298333336, 60.21245653833333, 60.92111866966667] [14.2370068  14.9984127  15.76371882 16.47238095] [15.28364323 16.06911565 16.85292517 17.67434618] 0.16744232177734375

seg 1 player combo 1 1 83
seg 1 player combo 1 1 0
[0.00000000e+00 3.02344785e-05 3.02341474e-05 1.51175828e-05]
      [44.448707483, 45.21449735466667, 45.964066515333336, 46.67466364366667] [0.         0.76575964 1.5153288  2.22594104] [0.         0.76578987 1.51535903 2.22595616] 0.14457464218139648

seg 1 player combo 1 1 20
[1.51172789e-05 1.51169206e-05 3.02344717e-05 3.02342857e-05]
      [58.685729403, 59.447135298333336, 60.21245653833333, 60.92111866966667] [14.2370068  14.9984127  15.76371882 16.47238095] [14.23702192 14.99842782 15.76374906 16.47241119] 0.14505743980407715

seg 1 player combo 1 2 83
seg 1 player combo 1 2 0
[ 0.         -0.02491308 -0.05928949 -0.05008314]
      [44.448707483, 45.2144973546

seg 1 player combo 4 2 20
[-0.59749055 -0.66817838 -0.56654573 -0.59752079]
      [64.72884353733333, 65.46297808, 66.17547996966667, 66.89857898700001] [14.47337868 15.20752834 15.92004535 16.64312925] [13.87588813 14.53934996 15.35349962 16.04560847] 0.10755300521850586

seg 1 player combo 4 2 40
[-0.9701285  -1.04003023 -1.06637944 -1.08390023]
      [78.95956160233332, 79.673318216, 80.39298563866666, 81.09490551766666] [28.70412698 29.41786848 30.13755102 30.83945578] [27.73399849 28.37783825 29.07117158 29.75555556] 0.07740211486816406

seg 1 player combo 4 3 83
seg 1 player combo 4 3 0
[ 0.          0.0240514  -0.00952381 -0.05235072]
      [50.255434618, 50.922101285000004, 51.64876795166666, 52.38787603933334] [0.         0.66666667 1.39333333 2.1324263 ] [0.         0.69071806 1.38380952 2.08007559] 0.14027857780456543

seg 1 player combo 4 3 40
[-3.31925926 -3.36861678 -3.44919123 -3.4861678 ]
      [78.95956160233332, 79.673318216, 80.39298563866666, 81.09490551766666] [28.

seg 0 player combo 1 0 40
[-5.25266818 -5.36627362 -5.43960695 -5.59945578]
      [38.54388510933333, 39.31720332566666, 40.05334845066667, 40.91312169333333] [32.44780045 33.22113379 33.95727891 34.81705215] [27.19513227 27.85486017 28.51767196 29.21759637] 0.09908723831176758

seg 0 player combo 1 0 60
[-7.26006047 -7.3985034  -7.48034769 -7.66692366]
      [54.804217687000005, 55.598820862000004, 56.378488284333336, 57.267513227333325] [48.19732426 48.99197279 49.77160998 50.66063492] [40.93726379 41.59346939 42.29126228 42.99371126] 0.08432626724243164

seg 0 player combo 1 1 77
seg 0 player combo 1 1 0
[ 0.00000000e+00  1.51171406e-05  1.51173016e-05 -1.51171043e-05]
      [6.606863189666666, 7.440166288666667, 8.266243386333334, 9.1] [0.         0.83328798 1.65936508 2.49315193] [0.         0.8333031  1.6593802  2.49313681] 0.15269160270690918

seg 0 player combo 1 1 20
[ 1.51174014e-05 -1.90475191e-10 -1.51169388e-05  1.51170113e-05]
      [22.733953137, 23.503053665666666, 24.2

KeyboardInterrupt: 

In [36]:
def realTimeAccompanimentWPrior(new_input,stored_info,evaluate, sim):
    #prior = stored_info['prior']
    orchestra_chroma = stored_info['orchestra_chroma']
    '''
    audio_input = np.zeros((len(new_input)+len(prior)))
    if len(prior)>0:
        audio_input[:len(prior)]=prior
        audio_input[len(prior):]=new_input
    else:
        audio_input=new_input
    '''
    prior = stored_info['prior']
    if len(prior)!=0:
        input_with_prior = np.zeros(len(prior)+len(new_input))
        input_with_prior[:len(prior)] = prior
        input_with_prior[len(prior):] = new_input
    else:
        input_with_prior = new_input
    stored_info['prior'] = []
    orchestra_audio = stored_info['orchestra_audio']
    input_chroma = getChromaFeatures(input_with_prior)
    cost_matrix = getCostMatrix(input_chroma, orchestra_chroma)
    D,wp = subsequencialDTW(cost_matrix)
    wp = np.sort(wp,axis=0)
    tf, itf, audio_output = [], [], []
    if evaluate:
        tf = group_time_function([wp],len(orchestra_audio))
    if sim:
        itf = inverse_time_function(wp)
        audio_output = TSM_Import.phase_vocoder(orchestra_audio, (itf).astype(int))
    
    #stored_info['prior'].append(new_input)
    return audio_output, tf, stored_info